# utility functions

In [49]:
import numpy as np
import random
import plotly.graph_objects as go
from numba import jit,prange

@jit
def oneStep(fitness,mF,n):
    
    deado = random.randint(0,n-1)
    #print('deado: ' + str(deado))
    if fitness[deado] == mF: numMutants = 1
    else: numMutants = 0
    if deado == 0:
        fitness[deado] = fitness[1]
        
    elif deado == n-1:
        fitness[deado] = fitness[n-2]
    else:
        fitLeft = fitness[deado-1]; fitRight = fitness[deado+1]
        pLeft = fitLeft/(fitLeft + fitRight)
        
        if np.random.rand() < pLeft:
            fitness[deado] = fitness[deado-1]
        else:
            fitness[deado] = fitness[deado+1]
            
    if fitness[deado] == mF and numMutants == 0: numMutants = 1
    elif (not fitness[deado] == mF) and numMutants == 1: numMutants = -1
    else: numMutants = 0
        
    return fitness,numMutants

@jit
def didItFix(n,mutant,mutantFitness):
    
    fitness = np.zeros(int(n))+1; fitness[int(mutant)] = mutantFitness
     
    numMutants = 1
    timeStep =1
    fixed = 0
    finished = False
    maxTime = 10000000
    
    
    while numMutants < n and numMutants > 0:
        fitness, newMutants = oneStep(fitness,mutantFitness,n)

        numMutants += newMutants; timeStep += 1      

        if numMutants == n: fixed = 1; break
        elif timeStep > maxTime:
                fitness = np.zeros(int(n))+1; fitness[int(mutant)] = mutantFitness
                numMutants = 1
                timeStep =1
            
    return fixed,timeStep

@jit
def fixationProbability(n,mutant,mutantFitness,numTrials):
    
    fixedItDid = np.zeros(numTrials)
    fixTimes = np.zeros(numTrials)
    
    for i in prange(numTrials):
        fix,time = didItFix(n,mutant,mutantFitness)
        fixedItDid[i] = fix; fixTimes[i] = time
    
    if np.sum(fixedItDid) > 0:
        fixers = fixTimes[np.where(fixedItDid == 1)]
    else: fixers = np.zeros(2)
        
    return np.mean(fixedItDid),np.mean(fixers)
    


def builderFigure(n,mutant, mutantFitness,runTime):
        
    X = np.arange(1,n+1)

    fitness = np.zeros(n)+1; fitness[mutant] = mutantFitness
    fitnessFirst = fitness
    time = np.repeat(0,n)
    
    numMutants = 1
    timeStep =1
    
    framelets = []
    fixTime = None
    
    while timeStep < runTime and numMutants < n:
        fitness, newMutants = oneStep(fitness,mutantFitness,n)
        
        numMutants += newMutants; timeStep += 1
        
        framelets.append(go.Frame(data = [go.Scatter(x = X, y = fitness,mode = 'markers')],
                                  layout = go.Layout(title_text = "step " + str(timeStep))))
        

        if numMutants == n: fixTime = timeStep
        if np.mod(timeStep,25000) == 0: print("time: " + str(timeStep))
            
    fig = go.Figure(
        data=[go.Scatter(x=X, y=fitnessFirst)],
        layout=go.Layout(
            xaxis=dict(title = 'node',range=[0, n+1], autorange=False),
            yaxis=dict(title = "fitness",range=[0.5, 2], autorange=False),
            title = "Fixation at timestep " + str(fixTime),
            updatemenus=[    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        
                            ]}]
        ),
        frames=framelets
    )

    fig.show()
    
    return fig

## Part A

In [34]:
from plotly.subplots import make_subplots
def comparePlotA(mutantFitness,numTrials):
    n = 101
    checkLocs = np.linspace(0,100,26)

    times = np.zeros(26)
    fixProbs = np.zeros(26)
    for i in range(26):
        fixProbs[i],times[i] = fixationProbability(n,checkLocs[i],mutantFitness,numTrials)

    fig = make_subplots(rows = 2, cols = 1,subplot_titles = ("fixation probabilities","fixation times"))
    fig.add_trace(go.Scatter(x = checkLocs+1,y = fixProbs,name = "fixation probabilities",mode = 'markers'),row = 1,col = 1)
    fig.add_trace(go.Scatter(x = checkLocs+1,y = times, name = "fixation times",mode = 'markers'),row = 2,col = 1)

    fig.update_yaxes(title = "probabilities", row = 1, col = 1)
    fig.update_yaxes(title = "time", row = 2, col = 1)
    fig.update_xaxes(title = "position", row = 2, col = 1)
    fig.update_layout(title_text = 'Mutant Fitness = ' + str(mutantFitness))
    fig.show()
    
    return fixProbs,times,checkLocs
    
def comparePlotAjustPlot(fixProbs,times,checkLocs):
    fig = make_subplots(rows = 2, cols = 1,subplot_titles = ("fixation probabilities","fixation times"))
    fig.add_trace(go.Scatter(x = checkLocs+1,y = fixProbs,name = "fixation probabilities",mode = 'markers'),row = 1,col = 1)
    fig.add_trace(go.Scatter(x = checkLocs+1,y = times, name = "fixation times",mode = 'markers'),row = 2,col = 1)

    fig.update_yaxes(title = "probabilities", row = 1, col = 1)
    fig.update_yaxes(title = "time", row = 2, col = 1)
    fig.update_xaxes(title = "position", row = 2, col = 1)
    fig.update_layout(title_text = 'Mutant Fitness = ' + str(mutantFitness))
    fig.show()

In [56]:
n = 11; mutantLoc = 5; mutantFit = 1.5; runTime = 1000
data = builderFigure(n,mutantLoc,mutantFit,runTime)

In [36]:
fixProbsA98,timesA98,checkLocsA98=comparePlotA(.98,100000)
#comparePlotAjustPlot(fixProbs,times,checkLocs)

In [14]:
fixProbsA102, timesA102, checkLocsA102 = comparePlotA(1.02,50000)

In [15]:
fixProbsA11, timesA11, checkLocsA11 = comparePlotA(1.1,50000)

## Part B

In [37]:
def comparePlotsB(mutantFitness,numTrials):
    n_array = [20,30,50,100,150,200]
    mutant_loc = [9,14,24,49,74,99]

    length = len(n_array)
    times = np.zeros(length)
    fixProbs = np.zeros(length)
    for i in range(length):
        fixProbs[i],times[i] = fixationProbability(n_array[i],mutant_loc[i],mutantFitness,numTrials)
        print("finished n = " + str(n_array[i]))

    rhos = [(1-1/mutantFitness)/(1-1/mutantFitness**N) for N in n_array]
    fig = make_subplots(rows = 2, cols = 1,subplot_titles = ("fixation probabilities","fixation times"))
    fig.add_trace(go.Scatter(x = n_array,y = fixProbs,name = "fixation probabilities",mode = 'markers'),row = 1,col = 1)
    fig.add_trace(go.Scatter(x = n_array,y = times, name = "fixation times",mode = 'markers'),row = 2,col = 1)
    fig.add_trace(go.Scatter(x = n_array, y = rhos, name = r"$\rho$", mode = 'markers'),row =1, col =1)

    fig.update_yaxes(title = "probabilities", row = 1, col = 1)
    fig.update_yaxes(title = "time", row = 2, col = 1)
    fig.update_xaxes(title = "n", row = 2, col = 1)
    fig.update_layout(title_text = 'Mutant Fitness = ' + str(mutantFitness))
    fig.show()
    
    return fixProbs,times,n_array

def comparePlotsBjustPlots(fixProbs,times,n_array,mutantFitness):
    rhos = [(1-1/mutantFitness)/(1-1/mutantFitness**N) for N in n_array]
    
    fig = make_subplots(rows = 2, cols = 1,subplot_titles = ("fixation probabilities","fixation times"))
    fig.add_trace(go.Scatter(x = checkLocs+1,y = fixProbs,name = "fixation probabilities",mode = 'markers'),row = 1,col = 1)
    fig.add_trace(go.Scatter(x = checkLocs+1,y = times, name = "fixation times",mode = 'markers'),row = 2,col = 1)
    fig.add_trace(go.Scatter(x = n_array, y = rhos, name = r"$\rho$", mode = 'markers'),row= 1, col =1)
    
    fig.update_yaxes(title = "probabilities", row = 1, col = 1)
    fig.update_yaxes(title = "time", row = 2, col = 1)
    fig.update_xaxes(title = "n", row = 2, col = 1)
    fig.update_layout(title_text = 'Mutant Fitness = ' + str(mutantFitness))
    fig.show()

In [39]:
fixProbsB98, timesB98, n_array = comparePlotsB(.98,10000)
#comparePlotsBjustPlots(fixProbsB98, timesB98, n_array,.98)

finished n = 20
finished n = 30
finished n = 50
finished n = 100
finished n = 150
finished n = 200


In [40]:
fixProbsB102, timesB102, n_arrayB102 = comparePlotsB(1.02,10000)

finished n = 20
finished n = 30
finished n = 50
finished n = 100
finished n = 150
finished n = 200


In [42]:
fixProbsB11, timesB11, n_arrayB11 = comparePlotsB(1.1,10000)

finished n = 20
finished n = 30
finished n = 50
finished n = 100
finished n = 150
finished n = 200


# utility functions for part d

In [42]:
import numpy as np
from numba import jit, prange
import plotly.graph_objects as go
@jit
def nowakOneStep(fitness,mF,n):
    
    numMutants = 0
    chanceArray = fitness/np.sum(fitness)
    rep = 0
    which = np.random.rand()
    
    # find index of thingy who replicates
    halfway = int(n/2)
    quarter = int(n/4)
    threeQuarters = quarter*3
    
    if which < np.sum(fitness[0:halfway]):
        if which < np.sum(chanceArray[0:quarter]):
            rep = findRep(which,chanceArray,start = 0)
        else:
            rep = findRep(which, chanceArray,start = quarter)
    else:
        if which < np.sum(chanceArray[0:threeQuarters]):
            rep = findRep(which, chanceArray,start = halfway)
        else:
            rep = findRep(which, chanceArray,start = threeQuarters)
            
    # replicate! 
    if rep == 0 or rep == n-1: lost = fitness[rep]; gained = fitness[rep]
    else:
        if rep <= halfway-1:
            lost = fitness[0]; gained = fitness[rep]
            #try:
            fitness[0:rep] = fitness[1:rep+1]
            #except:
            #    print("rep: " + str(rep))
            #    print(fitness)
                
        else:
            lost = fitness[-1]; gained = fitness[rep]
            fitness[rep+1:] = fitness[rep:-1]
    
    if lost == mF and not gained == mF: numMutants = -1
    elif gained == mF and not lost == mF: numMutants = 1
            
    return fitness, numMutants
    
            
@jit  
def findRep(which, fitness,start):
    
    it= start
    sumSoFar = np.sum(fitness[0:it+1])
 
    while which > sumSoFar:
        it += 1
        sumSoFar += fitness[it]
    max(it-1,0)
    return it
    

@jit
def nowakDidItFix(n,mutant,mutantFitness):
    
    fitness = np.zeros(int(n))+1; fitness[int(mutant)] = mutantFitness
     
    numMutants = int(1)
    timeStep =1
    fixed = int(-1)
    maxTime = 20000000
    while numMutants < n and numMutants > 0:
        fitness, newMutants = nowakOneStep(fitness,mutantFitness,n)
        
        numMutants += newMutants; timeStep += 1      

        if numMutants == n: fixed = int(1); break
        elif numMutants == 0: fixed = int(0); break
        elif timeStep > maxTime:
            fitness = np.zeros(int(n))+1; fitness[int(mutant)] = mutantFitness
            numMutants = int(1)
            timeStep =1
            
    return fixed,timeStep

@jit
def nowakFixationProbability(n,mutant,mutantFitness,numTrials):
    
    fixedItDid = np.zeros(numTrials)
    fixTimes = np.zeros(numTrials)
    
    for i in range(numTrials):
        fix,time = nowakDidItFix(n,mutant,mutantFitness)
        fixedItDid[i] = fix; fixTimes[i] = time
       
    if np.sum(fixedItDid) > 0:
        fixers = fixTimes[np.where(fixedItDid == 1)]
    else: fixers = np.zeros(2)
    return np.mean(fixedItDid),np.mean(fixers)

def nowakBuilderFigure(n,mutant, mutantFitness,runTime):
        
    X = np.arange(1,n+1)

    fitness = np.zeros(n)+1; fitness[mutant] = mutantFitness
    fitnessFirst = fitness
    time = np.repeat(0,n)
    
    numMutants = 1
    timeStep =1
    
    framelets = []
    fixTime = None
    
    while timeStep < runTime and numMutants < n:
        fitness, newMutants = nowakOneStep(fitness,mutantFitness,n)
        
        numMutants += newMutants; timeStep += 1
        
        framelets.append(go.Frame(data = [go.Scatter(x = X, y = fitness,mode = 'markers')],
                                  layout = go.Layout(title_text = "step " + str(timeStep))))
        

        if numMutants == n: fixTime = timeStep
        if np.mod(timeStep,5000) == 0: print("time: " + str(timeStep))
            
    fig = go.Figure(
        data=[go.Scatter(x=X, y=fitnessFirst)],
        layout=go.Layout(
            xaxis=dict(range=[0, n+1], autorange=False),
            yaxis=dict(range=[0.5, 2], autorange=False),
            title = "Fixation at timestep " + str(fixTime),
            updatemenus=[    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        
                            ]}]
        ),
        frames=framelets
    )

    fig.show()
    
    return fig

In [5]:
n = 5; mutant = 3; mutantFitness = 40
nowakDidItFix(n,mutant,mutantFitness)

(0, 77)

In [49]:
from plotly.subplots import make_subplots
def nowakComparePlot(mutantFitness,numTrials):
    n = 101
    checkLocs = np.linspace(0,100,26)


    times = np.zeros(26)
    fixProbs = np.zeros(26)
    for i in range(26):
        fixProbs[i],times[i] = nowakFixationProbability(n,checkLocs[i],mutantFitness,numTrials)

    fig = make_subplots(rows = 2, cols = 1,subplot_titles = ("fixation probabilities","fixation times"))
    fig.add_trace(go.Scatter(x = checkLocs+1,y = fixProbs,name = "fixation probabilities",mode = 'markers'),row = 1,col = 1)
    fig.add_trace(go.Scatter(x = checkLocs+1,y = times, name = "fixation times",mode = 'markers'),row = 2,col = 1)

    fig.update_yaxes(title = "probabilities", row = 1, col = 1)
    fig.update_yaxes(title = "time", row = 2, col = 1)
    fig.update_xaxes(title = "position", row = 2, col = 1)
    fig.update_layout(title_text = 'Mutant Fitness = ' + str(mutantFitness))
    fig.show()
    
    return fixProbs,times,checkLocs

# part d

In [59]:
fixProbsD98, timesD98, checkLocsD98 = nowakComparePlot(.98,1000)

In [60]:
fixProbsD102, timesD102, checkLocsD102 = nowakComparePlot(1.02,10000)

In [61]:
fixProbsD11, timesD11, checkLocsD11 = nowakComparePlot(1.1,10000)

# sample animation for part d

In [45]:
n = 11; mutantLoc = 3; mutantFit = 1.5; runTime = 1000
data = nowakBuilderFigure(n,mutantLoc,mutantFit,runTime)

# sample animation for part a

In [30]:
n = 11; mutantLoc = 5; mutantFit = 1.5; runTime = 1000
data = builderFigure(n,mutantLoc,mutantFit,runTime)
#data.show()